In [6]:
import pandas as pd
import numpy as np
import datetime
from nlp_workflow import nlp_workflow

In [7]:
df = pd.read_csv('food_wishes_from2012.csv')
df.head(3)

,Unnamed: 0,video_id,video_title,upload_date,view_count,like_count,comment_count
0,0,CQPLo8hECWg,Twice Baked Potatoes -- How to Make Fancy Stuf...,2012-12-15,4194592,47213,1950
1,1,TsrTU3CJn2c,Irish Shepherd's Pie - Classic Shepherd Pie fo...,2012-03-05,2730866,44230,2843
2,2,wRtGM3f-UBc,How to Flip Food in a Pan Like a Chef!,2012-10-04,2764323,17837,1382


In [8]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [15]:
df.dtypes

video_id                    object
video_title                 object
upload_date         datetime64[ns]
view_count                   int64
like_count                   int64
comment_count                int64
upload_year                  int64
upload_month                 int64
upload_day                   int64
upload_yearmonth         period[M]
duplicates                    bool
dtype: object

In [9]:
df['upload_date'] = pd.to_datetime(df['upload_date'], format='%Y-%m-%d')
df['upload_year'] = df['upload_date'].dt.year
df['upload_month'] = df['upload_date'].dt.month
df['upload_day'] = df['upload_date'].dt.day
df['upload_yearmonth'] = df['upload_date'].dt.to_period('M')

In [ ]:
df['duplicates'] = df['video_id'].duplicated(keep=False)
duplicates = df[df['duplicates'] == True]
duplicates

In [ ]:
df['video_id'].dtype

In [4]:
df = nlp_workflow(df=df, columns=['video_title'])
df.head(1)

,Unnamed: 0,video_id,video_title,upload_date,view_count,like_count,comment_count,video_title_nlp
0,0,CQPLo8hECWg,Twice Baked Potatoes -- How to Make Fancy Stuf...,2012-12-15,4194592,47213,1950,"[twice, bake, potato, make, fanci, stuf, potato]"
1,1,TsrTU3CJn2c,Irish Shepherd's Pie - Classic Shepherd Pie fo...,2012-03-05,2730866,44230,2843,"[irish, shepherd, pie, classic, shepherd, pie,..."
2,2,wRtGM3f-UBc,How to Flip Food in a Pan Like a Chef!,2012-10-04,2764323,17837,1382,"[flip, food, pan, like, chef]"


In [16]:
df.groupby('upload_yearmonth').sum().sort_values(by='view_count', ascending=False)['view_count']

upload_yearmonth
2013-04    21228400
2012-02    17862570
2017-04    12954284
2015-03    12722974
2015-06    12203355
             ...   
2021-07     1639196
2021-09     1600416
2020-11     1328946
2019-12     1321833
2022-03      530099
Freq: M, Name: view_count, Length: 123, dtype: int64